In [ ]:
import os
import cv2
import time
import scipy as sp
import numpy as np
import random as rn
import pandas as pd
from tqdm import tqdm
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt


import tensorflow as tf
import keras
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K
from keras.activations import elu
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Layer
from tensorflow.keras.utils import get_custom_objects
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import cohen_kappa_score

TRAIN_DF_PATH = "/kaggle/input/hackathon-thai-finger-spelling-recognition/train.csv"
TEST_DF_PATH = '/kaggle/input/hackathon-thai-finger-spelling-recognition/test.csv'
TRAIN_IMG_PATH = "/kaggle/input/hackathon-thai-finger-spelling-recognition/images/train"
TEST_IMG_PATH = '/kaggle/input/hackathon-thai-finger-spelling-recognition/images/test'
SAVED_MODEL_NAME = 'effnetV2L_model6.h5'

seed = 42
rn.seed(seed)
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# Data

In [ ]:
print("Image IDs and Labels (TRAIN)")
train_df = pd.read_csv(TRAIN_DF_PATH)

# Add extension to id_code
train_df['id'] = train_df['id']
print(f"Training images: {train_df.shape[0]}")
display(train_df.head())
print("Image IDs (TEST)")
test_df = pd.read_csv(TEST_DF_PATH)

# Add extension to id_code
test_df['id'] = test_df['id']
print(f"Testing Images: {test_df.shape[0]}")
display(test_df.head())

In [ ]:
# Specify image size
IMG_WIDTH = 512
IMG_HEIGHT = 512
CHANNELS = 3

# Preprocessing Data

In [ ]:
def get_preds_and_labels(model, generator):
    preds = []
    labels = []
    for _ in range(int(np.ceil(generator.samples / BATCH_SIZE))):
        x, y = next(generator)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()

In [ ]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_kappas = []
    def on_epoch_end(self, epoch, logs={}):
        y_pred, labels = get_preds_and_labels(model, val_generator)
        y_pred = np.rint(y_pred).astype(np.uint8).clip(0, 4)

        _val_kappa = cohen_kappa_score(labels, y_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)
        print(f"val_kappa: {round(_val_kappa, 4)}")
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save(SAVED_MODEL_NAME)
        return

In [ ]:
def crop_image_from_gray(img, tol=7):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): 
            return img 
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def preprocess_image(image, sigmaX=10):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    return image

In [ ]:
# Labels for training data
y_labels = train_df['label'].values

# EfficientNetModel

In [ ]:
# Load in EfficientNet
import keras.applications as models
effnet = tf.keras.applications.EfficientNetV2L(weights="imagenet",
                        include_top=False,
                        input_shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS))

# Build_model

In [ ]:
def build_model():
    model = Sequential()
    model.add(effnet)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(20, activation="softmax"))
    model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['acc']
             )
    print(model.summary())
    return model

# Initialize model
model = build_model()

In [ ]:
train_df['label'] = train_df['label'].astype(str)

BATCH_SIZE = 4

train_datagen = ImageDataGenerator(rotation_range=20,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.20,
                                   preprocessing_function=preprocess_image, 
                                   rescale=1 / 128.)



train_generator = train_datagen.flow_from_dataframe(train_df, 
                                                    x_col='id', 
                                                    y_col='label',
                                                    directory = TRAIN_IMG_PATH,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical', 
                                                    subset='training')

val_generator = train_datagen.flow_from_dataframe(train_df, 
                                                  x_col='id', 
                                                  y_col='label',
                                                  directory = TRAIN_IMG_PATH,
                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  subset='validation')

In [ ]:
train_df['label'].value_counts().sort_index().plot(kind="bar", 
                                                       figsize=(12,5), 
                                                       rot=0)
plt.title("TrainSet Label Distribution", 
          weight='bold', 
          fontsize=18)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Label", fontsize=17)
plt.ylabel("Frequency", fontsize=17);

In [ ]:
kappa_metrics = Metrics()

es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=10)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=2, 
                        verbose=1, 
                        mode='auto', 
                        min_delta=0.0001)

model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=50,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // BATCH_SIZE,
                    callbacks=[kappa_metrics, es, rlr])

# Prediction

In [ ]:
loaded_model = load_model('/kaggle/working/effnetV2L_model6.h5')

In [ ]:
test_generator = train_datagen.flow_from_dataframe(test_df, 
                                                    x_col='id', 
                                                    y_col=None,
                                                    directory = TEST_IMG_PATH,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    class_mode=None,shuffle=False)

num_tta = 15
tta_preds = []
for i in range(num_tta):
    preds = loaded_model.predict(test_generator,3000)
    tta_preds.append(preds)
    
avg_preds = tf.reduce_mean(tta_preds, axis=0)

In [ ]:
label_map = {0: 0, 1: 1, 2: 10, 3: 11, 4: 12, 5: 13, 6: 14, 7: 15, 8: 16, 9: 17,
             10: 18, 11: 19, 12: 2, 13: 3, 14: 4, 15: 5, 16: 6, 17: 7, 18: 8, 19: 9}

pred_labels = []
for label in np.argmax(avg_preds, axis=-1):
    if label in label_map:
        pred_labels.append(label_map[label])
    else:
        pred_labels.append('unknown')

# Save predictions to a CSV file
submission_df = pd.DataFrame({'id': test_df['id'], 'predict': pred_labels})
submission_df.to_csv('finger6-EfficientNetV2L.csv', index=False)